# Plot Time Series of Max Amplitudes
Script for making plots of time series of nodes and nearby permanent stations at intersections

In [1]:
using SeisIO, SeisNoise, Plots, Dates, CSV, DataFrames, JLD2, Statistics, Glob, AWSS3, AWSCore, SCEDC, Distributed, AbstractFFTs, Plots.PlotMeasures, HDF5
cd()
cd("Downloads/Seismo/amplitude_data")

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/julianschmitt/.julia/packages/Plots/SjqWU/src/backends/hdf5.jl", 162, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010097011f, Ptr{Nothing} @0x0000000100a09eb3, Ptr{Nothing} @0x0000000100a0afab, Ptr{Nothing} @0x0000000100a098df, Ptr{Nothing} @0x0000000100a09bbc, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x0000000100a231ce, Ptr{Nothing} @0x0000000100a221b1, Ptr{Nothing} @0x0000000100a229d1, Ptr{Nothing} @0x0000000100a230d6, Ptr{Nothing} @0x00000001009fbb6d, Ptr{Nothing} @0x0000000100a2427c, Ptr{Nothing} @0x0000000109a1ed4f, Ptr{Nothing} @0x0000000129a75bfe, Ptr{Nothing} @0x0000000100a0afcf, Ptr{Nothing} @0x0000000100a098df, Ptr{Nothing} @0x0000000100a09bbc, Base.InterpreterIP in top-level CodeInfo for Plots at statement 10, Ptr{Nothing} @0x0000000100a231ce, Ptr{Nothing} @0x0000000100a24074, Ptr{Nothing} @0x

In [2]:
# Sub functions
function mov_avg(df::DataFrame, range::Int64=30) 
    df_new = df[:,["date","max_amp"]]
    amps = df_new.max_amp
    avg, len = similar(amps), length(amps)
    for i =1:len
        first = max(1, convert(Int64, round(i-range/2)))
        last = min(len, convert(Int64, round(i+range/2)))
        window = amps[first:last]
        sd, mn = std(window), mean(window)
        avg[i] = mean(filter(x -> (x < mn + 3*sd) && (x > mn-3*sd), window))
    end
    df_new.moving_avg = avg
    return df_new
end
function get_node_files(source::String, deployment_filt::String, int_range=Array{Int64, 1})
    return filter(x -> occursin(source, x) & occursin(deployment_filt,x) & (parse(Int64, split(x, ".")[end-2][end-1:end]) in collect(int_range[1]:int_range[2])), nodes)
end

get_node_files (generic function with 2 methods)

In [7]:
# Plotting and wrapper function to select nodes to plot
plot_root = "/Users/julianschmitt/Desktop/SeisPlots2/"
function plot_tseries(perm_station::String, node_stations_::Array{String,1}, source_, dir::String)
    test = DataFrame(CSV.File(perm_station))
    #get moving averages and naming schemes
    times, amplitudes = test.date, test.max_amp
    times_avg, amps_avg = mov_avg(test,90).date, mov_avg(test,90).moving_avg
    perm_name = join(split(split(perm_station,"/")[end],".")[3:4],".")
    node_range = join([split(node_stations_[1],".")[end-2],split(node_stations_[end],".")[end-2][end-1:end]],"-")
    name = "$perm_name and $node_range Peak Amplitude Time Series"
    
    # Plot permanent station moving average
    tseries = plot(times_avg, amps_avg, title = name, label="$perm_name Mov Avg", color="gray", 
        linewidth=2, xtickfontsize=10, xlabel="Time", ylabel="Correlation Velocity Max Amp - filt: 0.1-0.5 Hz",
        legend = :outertopright, size=(900, 400), dpi=300, leftmargin= 7mm, bottommargin=5mm, linealpha=.6)
    
    # plot nodes
    for (ind, file) in enumerate(node_stations_)
        node_name, color = join(split(split(file,"/")[end],".")[3:4],"."), ""
        if occursin("NO.B1", node_name) # color B1 nodes green and intersecting nodes blue
            color = "green"
        else
            color = "blue"
        end
        test2 = DataFrame(CSV.File(file))
        test3 = mov_avg(test2, 20)
        times2, amplitudes2 = test2.date, test2.max_amp
        times3, amplitudes3 = test3.date, test3.moving_avg
        amplitudes3 .+= (mean(amps_avg)-mean(amplitudes3))
        plot!(tseries, times3, amplitudes3, color=color, linealpha=.6, linewidth=2, label="$node_name Mov Avg")
    end
    save_path = joinpath(plot_root, "tseries_scaled/$dir/CI.$(source_)_$(perm_name)_to_nodes.png")
    if !isdir(dirname(save_path))
        mkpath(dirname(save_path))
    end
    png(save_path)
end
function plot_simple_nodes(stations::Array{String,1}, source::String, vert_name::String, dir::String)
    name = "CI.$source Peak Amplitude Time Series at Intersect - $vert_name and B1"
    tseries = plot(title = name,linewidth=3, xtickfontsize=10, xlabel="Time", 
        ylabel="Correlation Velocity Max Amp - filt: 0.1-0.5 Hz",
        legend = :outertopright, size=(900, 400), dpi=300, leftmargin= 7mm, bottommargin=5mm)
    for (ind, file) in enumerate(stations)
        rec_name = join(split(split(file,"/")[end],".")[3:4],".")
        series = DataFrame(CSV.File(file))
        series_m = mov_avg(series, 5)
        times, amplitudes = series.date, series.max_amp
        times_m, amplitudes_m = series_m.date, series_m.moving_avg
        color=""
        if occursin("B1", rec_name)
            color = "green"
        else
            color = "blue"
        end
        println(rec_name)
        plot!(tseries, 1:length(amplitudes_m), amplitudes_m, color=color, linealpha=.6, 
                linewidth=2, label="$rec_name Mov Avg")
    end
    save_path = joinpath(plot_root,"tseries_scaled/$dir/CI.$(source)_B1_$(vert_name)_intersect.png")
    if !isdir(dirname(save_path))
        mkpath(dirname(save_path))
    end
    png(save_path)
end
function wrap_plotter(vnodes::Array{String,1}, bnodes::Array{String,1}, reciever::String, vert_name, dir)
    nodes = vcat(vnodes, bnodes)
    for source_ in ["SVD","CHN","CJM","DEV","IPT","LPC","SNO","TA2"]
        try
            nodes_source = filter(x -> occursin(source_, split(x, ".")[2]), readdir("Tseries_nodes"))
            node_intersect = filter(x-> any(occursin.(join(split(x,".")[3:4],"."), nodes)), nodes_source)

            plot_simple_nodes(joinpath.("Tseries_nodes",node_intersect), source_, vert_name, dir)
            rec = "Tseries/CI.$(source_).$reciever.csv"
            println(rec)
            nodes = ["Tseries_nodes/CI.$(source_).$(node).jld2.csv" for node in nodes]
            println(nodes)
            plot_tseries(rec, joinpath.("Tseries_nodes", node_intersect), source_, dir)
        catch e
            println(e)
        end
    end
end

wrap_plotter (generic function with 1 method)

In [4]:
# List nodes at the intersection
G2 = ["NO.G20$elt" for elt in collect(23:27)]
G1 = ["NO.G10$elt" for elt in collect(22:27)]
G3 = ["NO.G30$elt" for elt in collect(15:17)]
G4 = ["NO.G40$(lpad(elt,2,'0'))" for elt in collect(9:13)]
B2 = ["NO.B2$elt" for elt in collect(30:37)]
B3 = ["NO.B3$elt" for elt in collect(24:29)]
B4 = ["NO.B40$elt" for elt in collect(19:27)]
B5 = ["NO.B50$elt" for elt in collect(34:38)]
B6 = ["NO.B600$elt" for elt in collect(1:5)]
B1 = ["NO.B10$elt" for elt in vcat(collect(48:52), collect(14:18), collect(86:90))]
B1_1 = ["NO.B11$(lpad(elt,2,'0'))" for elt in vcat(collect(9:12), collect(91:95))]
B1_2 = ["NO.B12$(lpad(elt,2,'0'))" for elt in vcat(collect(57:60))]
#CI.CFS = B2/B6/B1 at end, CI.FON - B4, CI.RCU - close-ish to B3, CI.PDU - near B4, CI.RIO near G1, RUS bottom of G2, CAC - kinda far from G2
recievers = ["CI.CFS", "CI.FON", "CI.RCU", "CI.PDU", "CI.RUS", "CI.CAC"];

In [8]:
G1_int = [G1, ["NO.B10$elt" for elt in collect(12:21)], "CI.RIO", "G1", "B1_G1_intersect"] #012-021
G2_int = [G2, ["NO.B10$elt" for elt in collect(45:60)], "CI.RUS", "G2", "B1_G2_intersect"] #045-060
G3_int = [G3, ["NO.B10$elt" for elt in collect(83:92)], "CI.RIO", "G3", "B1_G3_intersect"] #083-092
G4_int = [G4,["NO.B11$elt" for elt in collect(10:16)], "CI.PSR", "G4", "B1_G4_intersect"] #110-116

#B1_int = [B1, ["NO.B11$elt" for elt in collect(45:60)], "CI.RUS", "B1", "B1_B1_intersect"]
B2_int = [B2, ["NO.B12$elt" for elt in collect(55:65)], "CI.CFS", "B2", "B1_B2_intersect"]
B3_int = [B3, ["NO.B11$elt" for elt in collect(90:96)], "CI.RCU", "B3", "B1_B3_intersect"]
B4_int = [B4, ["NO.B11$elt" for elt in collect(51:62)], "CI.PDU", "B4", "B1_B4_intersect"]
B5_int = [B5, ["NO.B11$elt" for elt in collect(16:24)], "CI.FON", "B5", "B1_B5_intersect"]
#B6_int = [B6, ["NO.B11$elt" for elt in collect(45:60)], "CI.CFS", "B6", "B1_B6_intersect"]
#to_plot = G2_int
for attrs in [B4_int, G2_int, B2_int, B3_int,G1_int, G3_int, G4_int, B2_int, B3_int, B5_int]
    wrap_plotter(attrs[1], attrs[2], attrs[3], attrs[4], attrs[5])
end

NO.B1257
NO.B1259
NO.B1260
Tseries/CI.CHN.CI.CFS.csv
["Tseries_nodes/CI.CHN.NO.B230.jld2.csv", "Tseries_nodes/CI.CHN.NO.B231.jld2.csv", "Tseries_nodes/CI.CHN.NO.B232.jld2.csv", "Tseries_nodes/CI.CHN.NO.B233.jld2.csv", "Tseries_nodes/CI.CHN.NO.B234.jld2.csv", "Tseries_nodes/CI.CHN.NO.B235.jld2.csv", "Tseries_nodes/CI.CHN.NO.B236.jld2.csv", "Tseries_nodes/CI.CHN.NO.B237.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1255.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1256.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1257.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1258.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1259.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1260.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1261.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1262.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1263.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1264.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1265.jld2.csv"]
ArgumentError("\"Tseries/CI.CHN.CI.CFS.csv\" is not a valid file")
NO.B1257
NO.B1259
NO.B1260
Tseries/CI.CJM.CI.CFS.csv
["Tseries_nodes/CI.CJM.Tseries_nodes

Tseries/CI.SNO.CI.CFS.csv
["Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B230.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B231.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B232.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B233.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B234.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.

["Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B230.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B231.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B232.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B233.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.c

Tseries/CI.LPC.CI.RCU.csv
["Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B324.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B325.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B326.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B327.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B328.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B329.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tserie

ArgumentError("\"Tseries/CI.TA2.CI.RCU.csv\" is not a valid file")
NO.B1014
NO.B1016
NO.B1017
NO.B1018
NO.G1022
NO.G1023
NO.G1024
NO.G1025
NO.G1026
NO.G1027
Tseries/CI.CHN.CI.RIO.csv
["Tseries_nodes/CI.CHN.NO.G1022.jld2.csv", "Tseries_nodes/CI.CHN.NO.G1023.jld2.csv", "Tseries_nodes/CI.CHN.NO.G1024.jld2.csv", "Tseries_nodes/CI.CHN.NO.G1025.jld2.csv", "Tseries_nodes/CI.CHN.NO.G1026.jld2.csv", "Tseries_nodes/CI.CHN.NO.G1027.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1012.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1013.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1014.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1015.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1016.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1017.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1018.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1019.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1020.jld2.csv", "Tseries_nodes/CI.CHN.NO.B1021.jld2.csv"]
NO.B1014
NO.B1016
NO.B1017
NO.B1018
NO.G1022
NO.G1023
NO.G1024
NO.G1025
NO.G1026
NO.G1027
Tseries/CI.CJM.CI.RIO.csv
["Tseries_nodes/CI.CJM.Tseri

...

Tseries/CI.IPT.CI.PDU.csv
["Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4019.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4020.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4021.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4022.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4023.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4024.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4025.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tserie

NO.B1155
NO.B1156
NO.B1157
NO.B1158
NO.B4021
NO.B4022
NO.B4023
NO.B4024
Tseries/CI.SVD.CI.PDU.csv
["Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4019.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4020.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4021.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4022.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_node

["Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4019.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4020.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4021.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B4022.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jl

Tseries/CI.IPT.CI.FON.csv
["Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5034.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5035.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5036.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5037.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5038.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B1116.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B1117.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tserie

Tseries/CI.TA2.CI.FON.csv
["Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5034.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5035.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5036.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld2.csv", "Tseries_nodes/CI.TA2.Tseries_nodes/CI.SVD.Tseries_nodes/CI.SNO.Tseries_nodes/CI.LPC.Tseries_nodes/CI.IPT.Tseries_nodes/CI.DEV.Tseries_nodes/CI.CJM.Tseries_nodes/CI.CHN.NO.B5037.jld2.csv.jld2.csv.jld2.csv.jld2.csv.jld

In [23]:
pwd()

"/Users/julianschmitt/Downloads/Seismo/amplitude_data"

In [39]:
function create_corr(file::HDF5File, station::String, stacktype::String, component::String)
    try
        S = CorrData()
        # get corr data
        corr = file[station][component][stacktype][1:end]
        S.corr = reshape(corr,size(corr,1),1)
        S.name = station
        S.dist = read(file[station]["meta"]["dist"])
        S.loc.lat = read(file[station]["meta"]["lat"])
        S.loc.lon = read(file[station]["meta"]["lon"])
        S.loc.el = read(file[station]["meta"]["el"])
        S.azi = read(file[station]["meta"]["azi"])
        S.baz = read(file[station]["meta"]["baz"])
        S.fs = 20. # For the 20 Hz product
        return S
    catch e
        #println(e)
    end
end

# get information for a set of filtered stations, default to all stations
function get_corrs(file::HDF5File, stacktype::String="linear",
    component::String="ZZ", filter::String="")
    filtered = [name for name in names(file) if occursin(filter, name)] 
    corrs = [create_corr(file, station, stacktype, component) for station in filtered]
    corrs = [corr for corr in corrs if !isnothing(corr)] # strip nothings - catches meta file etc which errors
    return corrs
end 
function fftderivative(A::CorrData)
    C = deepcopy(A)
    detrend!(C)
    taper!(C)
    F = fft(C.corr,1)
    F .*= fftfreq(length(C.corr)).* 1im .* 2π
    C.corr = real.(ifft(F,1))/(C.fs^2)
    return C
end
# gets location and maximum amplitude for corrs in array
function loc_max_amp(corr::CorrData, filt::Array{Float64,1}=[0.1,0.5])
    lat, lon = corr.loc.lat, corr.loc.lon
    bpass_corr = bandpass(fftderivative(fftderivative(corr)),filt[1],filt[2])
    half_len = convert(Int64,round(length(bpass_corr.corr)/2))
    max_ampl = maximum(bpass_corr.corr)
    max_ampl_anti = maximum(bpass_corr.corr[1:half_len])
    max_ampl_causal = maximum(bpass_corr.corr[half_len:end])
    snr = maximum(bpass_corr.corr)/std(bpass_corr.corr)
    snr_anti = maximum(bpass_corr.corr[1:half_len])/std(bpass_corr.corr[1:half_len])
    snr_causal = maximum(bpass_corr.corr[half_len:end])/std(bpass_corr.corr[half_len:end])
    return (lat, lon, max_ampl, max_ampl_anti, max_ampl_causal, snr, snr_anti, snr_causal, corr.name)
end

loc_max_amp (generic function with 2 methods)

In [41]:
year3 = 2019 # large stacked files have data between 2017 and 2019
filt = "" # return nodal plots
bandpass_filter, stacktype, components = [0.1, 0.35], "linear", ["ZZ","NN","EE"]
rootdir = "/Users/julianschmitt/Downloads/Seismo/"
for source_station in ["NO.B1259.--", "NO.B1260.--", "CI.SVD"]

    f2019 = h5open(joinpath(rootdir,"$(year3)/$(year3)_$(source_station).h5"),"r")
    for comp in components
        all_corrs = collect(Iterators.flatten([get_corrs(file, stacktype, comp, filt) for file in [f2019]]))
        amplitudes = [loc_max_amp(corr, bandpass_filter) for corr in all_corrs]
        pgd_amp = DataFrame(lat = [x[1] for x in amplitudes], lon = [x[2] for x in amplitudes],
            max_amp = [x[3] for x in amplitudes], max_amp_anti = [x[4] for x in amplitudes], 
            max_amp_causal = [x[5] for x in amplitudes],SNR = [x[6] for x in amplitudes], 
            SNR_anticausal = [x[7] for x in amplitudes], SNR_causal = [x[8] for x in amplitudes], station = [x[end] for x in amplitudes]);
        pgv_grouped = aggregate(groupby(pgd_amp, :station), mean)
        #println(pgd_amp)
        save_path = joinpath(rootdir,"amplitude_data_node/PGV_0.1_0.35/PGV_$(source_station)_$comp.csv")
        if !isdir(dirname(save_path))
            mkpath(dirname(save_path))
        end
        CSV.write(save_path ,pgv_grouped)
    end
end

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5O.c line 120 in H5Oopen(): unable to open object
    major: Object header
    minor: Can't open object
  #001: H5Oint.c line 596 in H5O__open_name(): unable to open object
    major: Object header
    minor: Can't open object
  #002: H5Oint.c line 551 in H5O_open_name(): object not found
    major: Object header
    minor: Object not found
  #003: H5Gloc.c line 422 in H5G_loc_find(): can't find object
    major: Symbol table
    minor: Object not found
  #004: H5Gtraverse.c line 851 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #005: H5Gtraverse.c line 627 in H5G__traverse_real(): traversal operator failed
    major: Symbol table
    minor: Callback failed
  #006: H5Gloc.c line 378 in H5G__loc_find_cb(): object 'ZZ' doesn't exist
    major: Symbol table
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5O.c line 120 in H5Oopen(